In [30]:
!pip install vaderSentiment langdetect emoji nltk vaderSentiment

StatementMeta(spark0623, 23, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=c42624c9e70dac6cf0e078ddcc17f0434c0317dd0e751da0fe971e43cd180c50
  Stored in directory: /home/trusted-service-user/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [31]:
from pyspark.sql.functions import udf,monotonically_increasing_id
from pyspark.sql.types import StringType, FloatType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pyspark.sql import functions as f  
import requests
import pandas as pd 
import datetime
import emoji
import nltk
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from notebookutils import mssparkutils

StatementMeta(spark0623, 23, 4, Finished, Available)

In [32]:
nltk.download('punkt')
nltk.download('stopwords')

StatementMeta(spark0623, 23, 5, Finished, Available)

[nltk_data] Downloading package punkt to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [33]:
Youtube_df = spark.read.parquet('abfss://files@datalakey4zmcph.dfs.core.windows.net/synapse/workspaces/data/youtube/youtube_cms/')

StatementMeta(spark0623, 23, 6, Finished, Available)

In [34]:
Youtube_df.show()

StatementMeta(spark0623, 23, 7, Finished, Available)

+--------------------+--------------------+-----------+--------------------+--------------------+--------+------+-----------+
|           Timestamp|            Username|    VideoID|             Comment|                Date|resource|symbol|collectedAt|
+--------------------+--------------------+-----------+--------------------+--------------------+--------+------+-----------+
|2024-06-04T02:17:43Z|      @jasonmraz3239|4L1ZV3Ql5-I|so what's ai pc? ...|2024-06-04T02:17:43Z| Youtube|   AMD| 2024-06-23|
|2024-06-04T01:38:44Z|        @rapatouille|4L1ZV3Ql5-I|wow didn't expect...|2024-06-04T01:38:44Z| Youtube|   AMD| 2024-06-23|
|2024-06-03T23:40:51Z|         @reichfrog2|4L1ZV3Ql5-I|No mention of RDN...|2024-06-03T23:40:51Z| Youtube|   AMD| 2024-06-23|
|2024-06-03T22:47:22Z|           @uros2909|4L1ZV3Ql5-I|I can't wait till...|2024-06-03T22:47:22Z| Youtube|   AMD| 2024-06-23|
|2024-06-03T22:21:38Z|             @eufefo|4L1ZV3Ql5-I|they forgot to la...|2024-06-03T22:21:38Z| Youtube|   AMD| 2024

In [35]:
Youtube_df_v1 = (
    Youtube_df.withColumn('wordCount', f.size(f.split(f.col('comment'), ' ')))
    .filter(f.col('wordCount')>5)
    .withColumn("id", monotonically_increasing_id())
    .select('Date','id','Comment','symbol','VideoID')
    ).toPandas()

StatementMeta(spark0623, 23, 8, Finished, Available)

In [36]:
Youtube_df_v1

StatementMeta(spark0623, 23, 9, Finished, Available)

,Date,id,Comment,symbol,VideoID
0,2024-06-04T02:17:43Z,0,so what's ai pc? what's the difference between...,AMD,4L1ZV3Ql5-I
1,2024-06-04T01:38:44Z,1,wow didn't expect the new CPUs launching so soon!,AMD,4L1ZV3Ql5-I
2,2024-06-03T23:40:51Z,2,No mention of RDNA 4 GPU's is disappointing. T...,AMD,4L1ZV3Ql5-I
3,2024-06-03T22:47:22Z,3,I can't wait till we get strix point to deskto...,AMD,4L1ZV3Ql5-I
4,2024-06-03T22:21:38Z,4,they forgot to launch an arm cpu.,AMD,4L1ZV3Ql5-I
...,...,...,...,...,...
13609,2024-06-04T14:17:11Z,60129543869,Is the reason why there are no ryzen 3s the ba...,AMD,G1aI7G9jevw
13610,2024-06-04T14:16:57Z,60129543870,The graphs at 7:25 are completely broken : how...,AMD,G1aI7G9jevw
13611,2024-06-04T14:14:57Z,60129543871,Imagine taking your kid to the playground to f...,AMD,G1aI7G9jevw
13612,2024-06-04T14:13:06Z,60129543872,Linus running through stats like a walk in the...,AMD,G1aI7G9jevw


In [37]:
# remove non english word
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False
    
# change text into lowercase
def to_lowercase(text):
    return text.lower()

# remove punctuation
def rm_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def tokenise(text):
    return word_tokenize(text)

# remove stop words 
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

# rejoin tokens
def rejoin_tokens(tokens):
    return ' '.join(tokens)

# main function for preprocess_text 
def preprocess_text(text):
    text = to_lowercase(text)
    text = rm_punctuation(text)
    tokens = tokenise(text)
    tokens = remove_stopwords(tokens)
    return rejoin_tokens(tokens)

StatementMeta(spark0623, 23, 10, Finished, Available)

In [38]:
Youtube_df_v1['is_english'] = Youtube_df_v1['Comment'].apply(is_english)
Youtube_df_v2 = Youtube_df_v1[Youtube_df_v1['is_english']]
Youtube_df_v2['preprocessed_text'] = Youtube_df_v2['Comment'].apply(preprocess_text)

StatementMeta(spark0623, 23, 11, Finished, Available)

/tmp/ipykernel_5378/2470966190.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Youtube_df_v2['preprocessed_text'] = Youtube_df_v2['Comment'].apply(preprocess_text)


In [39]:
def is_string(text):
    return isinstance(text, str)

Youtube_df_v2['is_string']=Youtube_df_v2['preprocessed_text'].apply(is_string)
Youtube_df_v3 = Youtube_df_v2[Youtube_df_v2['is_string']]

StatementMeta(spark0623, 23, 12, Finished, Available)

/tmp/ipykernel_5378/603027338.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Youtube_df_v2['is_string']=Youtube_df_v2['preprocessed_text'].apply(is_string)


In [40]:
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    return scores['compound']

def categorize_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

StatementMeta(spark0623, 23, 13, Finished, Available)

In [41]:
Youtube_df_v3['sentiment'] = Youtube_df_v3['preprocessed_text'].apply(analyze_sentiment)
Youtube_df_v3['sentiment_category'] = Youtube_df_v3['sentiment'].apply(categorize_sentiment)

StatementMeta(spark0623, 23, 14, Finished, Available)

In [42]:
Youtube_df_v4 = Youtube_df_v3[['id','sentiment','sentiment_category']]

StatementMeta(spark0623, 23, 15, Finished, Available)

In [43]:
final_df = pd.merge(Youtube_df_v1,Youtube_df_v4, left_on='id', right_on='id', how='left')

StatementMeta(spark0623, 23, 16, Finished, Available)

In [44]:
final_df

StatementMeta(spark0623, 23, 17, Finished, Available)

,Date,id,Comment,symbol,VideoID,is_english,sentiment,sentiment_category
0,2024-06-04T02:17:43Z,0,so what's ai pc? what's the difference between...,AMD,4L1ZV3Ql5-I,True,0.0000,neutral
1,2024-06-04T01:38:44Z,1,wow didn't expect the new CPUs launching so soon!,AMD,4L1ZV3Ql5-I,True,0.5859,positive
2,2024-06-03T23:40:51Z,2,No mention of RDNA 4 GPU's is disappointing. T...,AMD,4L1ZV3Ql5-I,True,-0.4939,negative
3,2024-06-03T22:47:22Z,3,I can't wait till we get strix point to deskto...,AMD,4L1ZV3Ql5-I,True,0.1531,positive
4,2024-06-03T22:21:38Z,4,they forgot to launch an arm cpu.,AMD,4L1ZV3Ql5-I,True,0.0000,neutral
...,...,...,...,...,...,...,...,...
13609,2024-06-04T14:17:11Z,60129543869,Is the reason why there are no ryzen 3s the ba...,AMD,G1aI7G9jevw,True,0.1010,positive
13610,2024-06-04T14:16:57Z,60129543870,The graphs at 7:25 are completely broken : how...,AMD,G1aI7G9jevw,True,-0.8070,negative
13611,2024-06-04T14:14:57Z,60129543871,Imagine taking your kid to the playground to f...,AMD,G1aI7G9jevw,True,-0.3818,negative
13612,2024-06-04T14:13:06Z,60129543872,Linus running through stats like a walk in the...,AMD,G1aI7G9jevw,True,0.3612,positive


In [45]:
result = final_df[['Date','symbol','Comment','sentiment','sentiment_category']]

StatementMeta(spark0623, 23, 18, Finished, Available)

In [46]:
result

StatementMeta(spark0623, 23, 19, Finished, Available)

,Date,symbol,Comment,sentiment,sentiment_category
0,2024-06-04T02:17:43Z,AMD,so what's ai pc? what's the difference between...,0.0000,neutral
1,2024-06-04T01:38:44Z,AMD,wow didn't expect the new CPUs launching so soon!,0.5859,positive
2,2024-06-03T23:40:51Z,AMD,No mention of RDNA 4 GPU's is disappointing. T...,-0.4939,negative
3,2024-06-03T22:47:22Z,AMD,I can't wait till we get strix point to deskto...,0.1531,positive
4,2024-06-03T22:21:38Z,AMD,they forgot to launch an arm cpu.,0.0000,neutral
...,...,...,...,...,...
13609,2024-06-04T14:17:11Z,AMD,Is the reason why there are no ryzen 3s the ba...,0.1010,positive
13610,2024-06-04T14:16:57Z,AMD,The graphs at 7:25 are completely broken : how...,-0.8070,negative
13611,2024-06-04T14:14:57Z,AMD,Imagine taking your kid to the playground to f...,-0.3818,negative
13612,2024-06-04T14:13:06Z,AMD,Linus running through stats like a walk in the...,0.3612,positive


In [48]:
spdf = spark.createDataFrame(result)
datalake_nm = 'datalake'+mssparkutils.env.getWorkspaceName()[7:] # get datalake name 
file_path ='abfss://files@{0}.dfs.core.windows.net/synapse/workspaces/data/youtube/youtube_sentiments'.format(datalake_nm)
spdf.write.parquet(file_path,mode='overwrite')

StatementMeta(spark0623, 23, 21, Finished, Available)